<img src="images/footfall_predictor.png">

In [1]:
import pandas as pd
import numpy as np
from datetime import date
import json

In [2]:
train = pd.read_csv("train_data.csv", na_values=["?", " ", "NA", "na"])
test = pd.read_csv("test_data.csv", na_values=["?", " ", "NA", "na"])

In [3]:
footfall=pd.read_csv("foot_fall.csv", sep=",", na_values=["?",",","#","NaN","unknown",""])
footfall

,city,2015-01-02,2015-01-03,2015-01-04,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-10,...,2018-06-21,2018-06-22,2018-06-23,2018-06-24,2018-06-25,2018-06-26,2018-06-27,2018-06-28,2018-06-29,2018-06-30
0,Ahmedabad,9384.0,6808.0,8064.0,10248.0,11180.0,6876.0,6436.0,7104.0,5924.0,...,9624.0,10476.0,9584.0,11940.0,13040.0,12144.0,10164.0,9784.0,10472.0,10040.0
1,Bangalore,16644.0,14640.0,15660.0,19056.0,19740.0,14280.0,12740.0,13480.0,11928.0,...,12412.0,13392.0,11948.0,13860.0,16384.0,16192.0,12388.0,12124.0,13412.0,12172.0
2,Chennai,19544.0,13752.0,13736.0,19740.0,21604.0,12820.0,12244.0,11956.0,10508.0,...,10420.0,11044.0,10320.0,11988.0,15248.0,15188.0,10596.0,9716.0,10776.0,10336.0
3,Delhi,16832.0,13256.0,14520.0,17324.0,17672.0,13356.0,12408.0,13072.0,10788.0,...,12308.0,11772.0,11492.0,13284.0,16224.0,15688.0,11728.0,11164.0,12272.0,11692.0
4,Hyderabad,13588.0,11548.0,11600.0,16336.0,18364.0,10788.0,9280.0,9264.0,8848.0,...,8832.0,8756.0,8528.0,9464.0,13356.0,14384.0,8900.0,7792.0,8420.0,9240.0
5,Jaipur,13000.0,11616.0,11848.0,12240.0,10992.0,11072.0,10556.0,11208.0,10180.0,...,9676.0,10496.0,9556.0,10500.0,11292.0,10544.0,9632.0,9656.0,10316.0,9652.0
6,Kolkata,13948.0,12104.0,12752.0,14492.0,14360.0,11984.0,11304.0,11728.0,10596.0,...,11700.0,11884.0,10488.0,12116.0,13696.0,13348.0,11576.0,10752.0,11896.0,10528.0
7,Lucknow,12308.0,9228.0,10792.0,13836.0,13188.0,9484.0,9004.0,8912.0,8208.0,...,8588.0,7640.0,7812.0,8676.0,11640.0,10536.0,8136.0,7644.0,7860.0,7880.0
8,Mumbai,19284.0,14472.0,16676.0,19684.0,19700.0,15484.0,14372.0,16024.0,13292.0,...,14068.0,15404.0,14088.0,16368.0,18800.0,18640.0,14772.0,14208.0,16044.0,14628.0
9,Other,8572.0,7180.0,6716.0,8616.0,9252.0,6852.0,6220.0,6808.0,5688.0,...,6512.0,6716.0,6164.0,6980.0,8056.0,8480.0,6480.0,6004.0,7144.0,6040.0


In [4]:
footfall = footfall.melt(id_vars=["city"],value_name="footfall",var_name="date")
footfall

,city,date,footfall
0,Ahmedabad,2015-01-02,9384.0
1,Bangalore,2015-01-02,16644.0
2,Chennai,2015-01-02,19544.0
3,Delhi,2015-01-02,16832.0
4,Hyderabad,2015-01-02,13588.0
...,...,...,...
12665,Jaipur,2018-06-30,9652.0
12666,Kolkata,2018-06-30,10528.0
12667,Lucknow,2018-06-30,7880.0
12668,Mumbai,2018-06-30,14628.0


In [5]:
#add footfall to the train and test
footfall[footfall["footfall"].isna()]

,city,date,footfall
10895,Jaipur,2018-01-02,NaN
10896,Kolkata,2018-01-02,NaN
10899,Other,2018-01-02,NaN


In [6]:
train

,year,month,day,city,medicine,sales
0,2015,1,2,1,1,24.0
1,2015,1,2,1,2,144.0
2,2015,1,2,1,3,84.0
3,2015,1,2,1,4,24.0
4,2015,1,2,1,5,8.0
...,...,...,...,...,...,...
22624340,2018,6,30,10,3056,8.0
22624341,2018,6,30,10,3163,16.0
22624342,2018,6,30,10,3164,8.0
22624343,2018,6,30,10,3157,4.0


In [7]:
test_train = pd.concat([train[["year", "month", "day", "city", "medicine"]], test[["year", "month", "day", "city", "medicine"]]])
test_train

,year,month,day,city,medicine
0,2015,1,2,1,1
1,2015,1,2,1,2
2,2015,1,2,1,3
3,2015,1,2,1,4
4,2015,1,2,1,5
...,...,...,...,...,...
773530,2018,7,31,10,3163
773531,2018,7,31,10,3164
773532,2018,7,31,10,3157
773533,2018,7,31,10,3165


In [8]:
def add_date(df): #faster way to stitch year month and day together into date
    df["date"] =  list(zip(df["year"], df["month"], df["day"]))
    df["date"] = df["date"].astype("category")
    (year, month, day) = zip(*(df["date"].cat.categories))
    new_cats = [str(year[i])+"-"+str(month[i])+"-"+str(day[i]) for i in range(0, len(day))]
    new_cats_map = dict(zip(df["date"].cat.categories, new_cats))
    df["date"]=df["date"].map(new_cats_map)
    df["date"] = pd.to_datetime(df["date"])
    #df.drop(["year", "month", "day"], axis=1, inplace=True)
    return df

In [9]:
test_train = add_date(test_train)
test_train

,year,month,day,city,medicine,date
0,2015,1,2,1,1,2015-01-02
1,2015,1,2,1,2,2015-01-02
2,2015,1,2,1,3,2015-01-02
3,2015,1,2,1,4,2015-01-02
4,2015,1,2,1,5,2015-01-02
...,...,...,...,...,...,...
773530,2018,7,31,10,3163,2018-07-31
773531,2018,7,31,10,3164,2018-07-31
773532,2018,7,31,10,3157,2018-07-31
773533,2018,7,31,10,3165,2018-07-31


In [10]:
test_train = test_train.drop(["year", "month", "day"], axis = 1)
test_train

,city,medicine,date
0,1,1,2015-01-02
1,1,2,2015-01-02
2,1,3,2015-01-02
3,1,4,2015-01-02
4,1,5,2015-01-02
...,...,...,...
773530,10,3163,2018-07-31
773531,10,3164,2018-07-31
773532,10,3157,2018-07-31
773533,10,3165,2018-07-31


In [9]:
from sklearn.preprocessing import OneHotEncoder

#this return a modified copy of df_test. It doesnot change the input dfs.
def onehotencode_column(df_train, df_test, col_name, remove_original=False, remove_redundant=False):
    onehotencoder = OneHotEncoder(handle_unknown='ignore')
    #create a copies of input df's for modification
    df_train_copy = df_train.copy()
    df_test_copy = df_test.copy()
    
    #ensure type of col_name is set as "category"
    df_train_copy[col_name] = df_train_copy[col_name].astype("category")
    df_test_copy[col_name] = df_test_copy[col_name].astype("category")

    #learn from df_train_copy
    onehotencoder = onehotencoder.fit(df_train_copy.loc[:,[col_name]])
    ohe_cat_col_names = onehotencoder.get_feature_names([col_name])
    
    #modify df_test_copy
    onehotencoded = onehotencoder.transform(df_test_copy.loc[:,[col_name]]).toarray()
    onehotencoded = pd.DataFrame(onehotencoded, columns=ohe_cat_col_names)
    df_test_copy = pd.concat([df_test_copy,onehotencoded],axis=1)
    
    #remove original column if asked to remove
    if remove_original == True:
        df_test_copy.drop(col_name, axis=1, inplace=True)
        
    #remove one redundant column created by one hot encoding if asked to remove
    if remove_redundant == True:
        df_test_copy.drop(ohe_cat_col_names[0], axis=1, inplace=True)
    return df_test_copy


In [13]:
def get_medicine_vector(name, gp, df):
    group = gp.copy()
    group = onehotencode_column(df, group, "medicine", True)
    group = pd.DataFrame(group.nunique() - 1).transpose()
    group = group.drop(["index"], axis = 1)
    group["date"] = pd.to_datetime(group["date"])
    group.loc[0, "date"] = name[1]
    group.loc[0, "city"] = name[0]
    return group

In [33]:
def get_city_date_from_groups(groups, df):
    for name, group in groups:
        g1 = group.reset_index()
        row_df = get_medicine_vector(name, g1, df)
        with open('footfall_metdata.csv', mode='a') as f:
            row_df.to_csv(f, index=False, header=f.tell()==0)
    return g

In [34]:
def get_city_date_from_df(df):
    city_date_groups = df.groupby(["city", "date"])
    return get_city_date_from_groups(city_date_groups, df)

In [35]:
get_city_date_from_df(test_train)

""


In [79]:
footfall_metdata = pd.read_csv("footfall_metdata.csv")


In [80]:
footfall

,city,date,footfall
0,Ahmedabad,2015-01-02,9384.0
1,Bangalore,2015-01-02,16644.0
2,Chennai,2015-01-02,19544.0
3,Delhi,2015-01-02,16832.0
4,Hyderabad,2015-01-02,13588.0
...,...,...,...
12665,Jaipur,2018-06-30,9652.0
12666,Kolkata,2018-06-30,10528.0
12667,Lucknow,2018-06-30,7880.0
12668,Mumbai,2018-06-30,14628.0


In [82]:
with open("city_dict.json") as f: cities = json.load(f)
city_dict = {int(key):value for key, value in cities.items()}

In [83]:
city_dict_rev = {value:int(key) for key, value in cities.items()}
city_dict_rev

{'Mumbai': 4,
 'Delhi': 5,
 'Bangalore': 7,
 'Kolkata': 1,
 'Chennai': 6,
 'Hyderabad': 8,
 'Ahmedabad': 9,
 'Jaipur': 3,
 'Lucknow': 10,
 'Other': 2}

In [84]:
footfall["city"]=footfall["city"].astype("category")
footfall["city"]=footfall["city"].map(city_dict_rev)

In [85]:
footfall

,city,date,footfall
0,9,2015-01-02,9384.0
1,7,2015-01-02,16644.0
2,6,2015-01-02,19544.0
3,5,2015-01-02,16832.0
4,8,2015-01-02,13588.0
...,...,...,...
12665,3,2018-06-30,9652.0
12666,1,2018-06-30,10528.0
12667,10,2018-06-30,7880.0
12668,4,2018-06-30,14628.0


In [93]:
footfall_merged = pd.merge(footfall, footfall_metdata, how="outer", left_on=["city", "date"],
                           right_on=["city", "date"])

In [94]:
footfall_merged.isna().sum()

city               0
date               0
footfall         333
medicine_1         0
medicine_2         0
                ... 
medicine_3419      0
medicine_3420      0
medicine_3421      0
medicine_3422      0
medicine_3423      0
Length: 3426, dtype: int64

In [95]:
footfall_merged

,city,date,footfall,medicine_1,medicine_2,medicine_3,medicine_4,medicine_5,medicine_6,medicine_7,...,medicine_3414,medicine_3415,medicine_3416,medicine_3417,medicine_3418,medicine_3419,medicine_3420,medicine_3421,medicine_3422,medicine_3423
0,9,2015-01-02,9384.0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,7,2015-01-02,16644.0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,6,2015-01-02,19544.0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,5,2015-01-02,16832.0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,8,2015-01-02,13588.0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12995,10,2018-07-27,NaN,1,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
12996,10,2018-07-28,NaN,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
12997,10,2018-07-29,NaN,1,1,1,0,1,1,1,...,1,0,0,0,0,0,0,0,0,0
12998,10,2018-07-30,NaN,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [99]:
def split_date(dataframe, date_col):
    df = dataframe.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df["year"] = df[date_col].dt.year
    df["month"] = df[date_col].dt.month
    df["week"] = df[date_col].dt.week
    df["day"] = df[date_col].dt.day
    df['quarter'] = df[date_col].dt.quarter
    df["day_of_week"] = df[date_col].dt.weekday
    df["weekend"] = (df["day_of_week"] > 5).astype(int)
    return df

In [101]:
footfall_merged = split_date(footfall_merged, "date")

In [102]:
footfall_merged = footfall_merged.drop(["date"], axis = 1)

In [130]:
ff_train = footfall_merged[~footfall_merged["footfall"].isnull()]#this effectively drops all the rows with null at any column

In [131]:
ff_train

,city,footfall,medicine_1,medicine_2,medicine_3,medicine_4,medicine_5,medicine_6,medicine_7,medicine_8,...,medicine_3421,medicine_3422,medicine_3423,year,month,week,day,quarter,day_of_week,weekend
0,9,9384.0,1,1,1,1,1,1,1,1,...,0,0,0,2015,1,1,2,1,4,0
1,7,16644.0,1,1,1,1,1,1,1,1,...,0,0,0,2015,1,1,2,1,4,0
2,6,19544.0,1,1,1,1,1,1,1,1,...,0,0,0,2015,1,1,2,1,4,0
3,5,16832.0,1,1,1,1,1,1,1,0,...,0,0,0,2015,1,1,2,1,4,0
4,8,13588.0,1,1,1,1,1,1,1,1,...,0,0,0,2015,1,1,2,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12665,3,9652.0,1,1,1,1,1,1,1,0,...,0,0,0,2018,6,26,30,2,5,0
12666,1,10528.0,1,1,1,1,1,1,1,1,...,0,0,0,2018,6,26,30,2,5,0
12667,10,7880.0,1,1,1,1,1,1,1,1,...,0,0,0,2018,6,26,30,2,5,0
12668,4,14628.0,1,1,1,1,1,1,1,1,...,0,0,0,2018,6,26,30,2,5,0


In [132]:
X_ff_train = ff_train.drop(["footfall"], axis = 1)
y_ff_train = ff_train["footfall"]

In [133]:
X_ff_train

,city,medicine_1,medicine_2,medicine_3,medicine_4,medicine_5,medicine_6,medicine_7,medicine_8,medicine_9,...,medicine_3421,medicine_3422,medicine_3423,year,month,week,day,quarter,day_of_week,weekend
0,9,1,1,1,1,1,1,1,1,0,...,0,0,0,2015,1,1,2,1,4,0
1,7,1,1,1,1,1,1,1,1,1,...,0,0,0,2015,1,1,2,1,4,0
2,6,1,1,1,1,1,1,1,1,1,...,0,0,0,2015,1,1,2,1,4,0
3,5,1,1,1,1,1,1,1,0,1,...,0,0,0,2015,1,1,2,1,4,0
4,8,1,1,1,1,1,1,1,1,1,...,0,0,0,2015,1,1,2,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12665,3,1,1,1,1,1,1,1,0,1,...,0,0,0,2018,6,26,30,2,5,0
12666,1,1,1,1,1,1,1,1,1,0,...,0,0,0,2018,6,26,30,2,5,0
12667,10,1,1,1,1,1,1,1,1,1,...,0,0,0,2018,6,26,30,2,5,0
12668,4,1,1,1,1,1,1,1,1,1,...,0,0,0,2018,6,26,30,2,5,0


In [134]:
y_ff_train

0         9384.0
1        16644.0
2        19544.0
3        16832.0
4        13588.0
          ...   
12665     9652.0
12666    10528.0
12667     7880.0
12668    14628.0
12669     6040.0
Name: footfall, Length: 12667, dtype: float64

In [137]:
X_ff_test = footfall_merged[footfall_merged["footfall"].isnull()]

In [138]:
X_ff_test = X_ff_test.drop(["footfall"], axis = 1)


In [139]:
X_ff_test

,city,medicine_1,medicine_2,medicine_3,medicine_4,medicine_5,medicine_6,medicine_7,medicine_8,medicine_9,...,medicine_3421,medicine_3422,medicine_3423,year,month,week,day,quarter,day_of_week,weekend
10895,3,1,1,1,1,1,1,1,1,1,...,0,0,0,2018,1,1,2,1,1,0
10896,1,1,1,1,1,0,1,1,1,1,...,0,0,0,2018,1,1,2,1,1,0
10899,2,1,1,1,1,0,1,1,1,0,...,0,0,0,2018,1,1,2,1,1,0
12670,1,1,1,1,1,1,1,1,1,0,...,0,0,0,2018,1,1,3,1,2,0
12671,1,1,1,1,1,1,1,1,1,1,...,0,0,0,2018,1,1,4,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12995,10,1,1,1,0,0,1,1,1,1,...,0,0,0,2018,7,30,27,3,4,0
12996,10,0,1,1,1,1,1,1,1,1,...,0,0,0,2018,7,30,28,3,5,0
12997,10,1,1,1,0,1,1,1,1,1,...,0,0,0,2018,7,30,29,3,6,1
12998,10,1,1,1,1,1,1,1,1,1,...,0,0,0,2018,7,31,30,3,0,0


In [146]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

reg_dt = DecisionTreeRegressor()
dt_param_grid = {
    'criterion': ["mse"],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'splitter': ["best"]
}
dt_grid_reg = GridSearchCV(reg_dt, param_grid=dt_param_grid, cv=5, n_jobs=-1, verbose=10)

In [147]:
dt_grid_reg.fit(X_ff_train, y_ff_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   36.5s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   48.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  43 out of  45 | elapsed:  2.2min remaining:    6.1s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  2.3min finished


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'criterion': ['mse'], 'min_samples_leaf': [1, 5, 10],
                         'min_samples_split': [2, 10, 20],
                         'splitter': ['best']},
             verbose=10)

In [153]:
y_ff_test = dt_grid_reg.predict(X_ff_test)

In [172]:
train_pred = dt_grid_reg.predict(X_ff_train.drop(["footfall"], axis = 1))

In [179]:
from sklearn.utils.validation import check_array
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = check_array([y_true, y_pred])

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [180]:
mean_absolute_percentage_error(y_ff_train, train_pred)

4.171357439752487

In [154]:
y_ff_test

array([ 9858.28571429, 14853.        ,  9350.66666667, 14853.        ,
       12436.33333333, 11197.33333333, 13573.88235294, 13920.        ,
       12818.5       , 12462.73684211, 12272.8       , 13648.57142857,
       12436.33333333, 12818.5       , 12694.33333333, 11329.        ,
       11544.        , 12908.5       , 11605.55555556, 11197.33333333,
       14580.88888889, 12818.5       , 13920.        , 11960.70588235,
       11414.18181818, 11414.18181818, 10872.        , 14580.88888889,
       12818.5       , 13546.28571429, 11414.18181818, 12389.6       ,
       10925.33333333, 11906.31578947, 12818.5       , 13542.46153846,
        9150.25      ,  7150.5       ,  6863.46666667,  7707.42857143,
        8625.68421053,  6786.75      ,  6840.        ,  6882.4       ,
        7112.        ,  6401.        ,  7598.4       ,  8149.42857143,
        7452.        ,  7360.        ,  6281.33333333,  7112.        ,
        6520.        ,  7598.4       ,  8625.68421053,  6738.75      ,
      

In [155]:
X_ff_test["footfall"] = y_ff_test

In [156]:
X_ff_test

,city,medicine_1,medicine_2,medicine_3,medicine_4,medicine_5,medicine_6,medicine_7,medicine_8,medicine_9,...,medicine_3422,medicine_3423,year,month,week,day,quarter,day_of_week,weekend,footfall
10895,3,1,1,1,1,1,1,1,1,1,...,0,0,2018,1,1,2,1,1,0,9858.285714
10896,1,1,1,1,1,0,1,1,1,1,...,0,0,2018,1,1,2,1,1,0,14853.000000
10899,2,1,1,1,1,0,1,1,1,0,...,0,0,2018,1,1,2,1,1,0,9350.666667
12670,1,1,1,1,1,1,1,1,1,0,...,0,0,2018,1,1,3,1,2,0,14853.000000
12671,1,1,1,1,1,1,1,1,1,1,...,0,0,2018,1,1,4,1,3,0,12436.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12995,10,1,1,1,0,0,1,1,1,1,...,0,0,2018,7,30,27,3,4,0,8291.250000
12996,10,0,1,1,1,1,1,1,1,1,...,0,0,2018,7,30,28,3,5,0,8140.222222
12997,10,1,1,1,0,1,1,1,1,1,...,0,0,2018,7,30,29,3,6,1,8518.105263
12998,10,1,1,1,1,1,1,1,1,1,...,0,0,2018,7,31,30,3,0,0,10318.400000


In [157]:
X_ff_train["footfall"] = y_ff_train

In [160]:
footfall_compact = pd.concat([
                                ff_train[["year", "month", "day", "city", "footfall"]],
                                X_ff_test[["year", "month", "day", "city", "footfall"]]
                             ])

In [163]:
footfall_compact = footfall_compact.sort_values(["year", "month", "day", "city"])

In [164]:
footfall_compact.to_csv("footfall_compact.csv", index=False)

In [166]:
footfall_complete = pd.concat([X_ff_train, X_ff_test])

In [167]:
footfall_complete.to_csv("footfall_complete.csv", index = False)